In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import numpy as np

def oecd_to_df(xml, data):
  # Parse the XML
  tree = ET.ElementTree(ET.fromstring(xml))
  root = tree.getroot()
  for obs in root.findall('.//{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic}Obs'):
    obs_key = obs.find('{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic}ObsKey')

    # Extract values from each <Value> element in <ObsKey>
    values = {value.attrib['id']: value.attrib['value'] for value in obs_key.findall('.//{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic}Value')}

    # Store the extracted data
    for key in data:
      if key is not "ObsValue":
        data[key].append(values[key])

    data['ObsValue'].append(obs.find('.//{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic}ObsValue').attrib['value'])

  # Create a DataFrame from the extracted data
  df = pd.DataFrame(data)
  return df

<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-55-019a468ab9d5>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if key is not "ObsValue":


In [ ]:
## One Time_Period
data_trust = {
    'TIME_PERIOD': [],
    'FREQ': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'SECTOR': [],
    'EDITION': [],
    'SCALE': [],
    'CATEGORY': [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.GOV.GIP,DSD_GOV_TDG_SPS_GPC%40DF_GOV_TDG_2023,1.0/A..TRUST_LE+TRUST_CL+TRUST_NG+SAD.....?dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text

df_trust = oecd_to_df(xml, data_trust)

df_trust = df_trust[['TIME_PERIOD', 'REF_AREA', 'MEASURE', 'ObsValue']]

reshaped_df_trust = df_trust.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    columns='MEASURE',
    values='ObsValue',
).reset_index()

reshaped_df_trust = reshaped_df_trust.rename(columns={'SAD' : 'Satisfaction_with_Democracy', 'TRUST_NG' : 'Trust_in_National_Government', 'TRUST_LE' : 'Trust_in_Legislature', 'TRUST_CL' : 'Trust_in_Court_and_Legal_System'})

data_gender = {
    'TIME_PERIOD': [],
    'FREQ': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'SECTOR': [],
    'EDITION': [],
    'CATEGORY': [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.GOV.GIP,DSD_GOV%40DF_GOV_EMPPS_REP_YU,1.0/A...PT_W...?startPeriod=2021&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_gender = oecd_to_df(xml, data_gender)

df_gender = df_gender[["REF_AREA", 'MEASURE', "ObsValue"]]

reshaped_df_gender = df_gender.pivot_table(
    index=['REF_AREA'],
    columns='MEASURE',
    values='ObsValue',
).reset_index()

reshaped_df_gender = reshaped_df_gender.rename(columns={'EMPW_PARL' : 'Gender_Equality_in_Parliament', 'EMPW_SMP' : 'Gender_equality_in_Senior_Management_Positions'})

data_demography = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'AGE' : [],
    'SEX' : [],
    'TIME_HORIZ' : [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_POPULATION%40DF_POP_HIST,1.0/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+G20+EU27+OECD..PS._T.Y_LE4+Y5T9+Y10T14+Y15T19+Y20T24+Y25T29+Y30T34+Y35T39+Y40T44+Y45T49+Y50T54+Y55T59+Y60T64+Y65T69+Y70T74+Y75T79+Y80T84+Y_GE85.?startPeriod=2021&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_demo = oecd_to_df(xml, data_demography)

reshaped_df_demo = df_demo.pivot_table(
    index=['REF_AREA'],
    columns='AGE',
    values='ObsValue',
    aggfunc='first'
)

age_groups = list(reshaped_df_demo.columns)

index_Y_LE4 = age_groups.index('Y_LE4')
index_Y5T9 = age_groups.index('Y5T9')
reordered_age_groups = [age_groups[index_Y_LE4], age_groups[index_Y5T9]] + age_groups[:index_Y5T9] + age_groups[index_Y5T9+1:index_Y_LE4] + age_groups[index_Y_LE4+1:]

reorderd_df_demo = reshaped_df_demo[reordered_age_groups]

# Adding the first 4 columns and storing the value in a new column
reorderd_df_demo['Under_20'] = reorderd_df_demo.iloc[:, 0:4].astype(float).sum(axis=1)
# Adding last 4 but excluding new Sum
reorderd_df_demo['Over_65'] = reorderd_df_demo.iloc[:, -5:-1].astype(float).sum(axis=1)
# Excludes the sum columns
reorderd_df_demo['Population'] = reorderd_df_demo.iloc[:, :-2].astype(float).sum(axis=1)
reorderd_df_demo['Percent_Under_20'] = reorderd_df_demo['Under_20'].astype(float) / reorderd_df_demo['Population'].astype(float)
reorderd_df_demo['Percent_Over_65'] = reorderd_df_demo['Over_65'].astype(float) / reorderd_df_demo['Population'].astype(float)


### Over time
data_life_exp = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'FREQ': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'AGE' : [],
    'SEX' : [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.ELS.HD,DSD_HEALTH_STAT@DF_LE,1.0/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA.A.LFEXP..Y0........?startPeriod=1980&endPeriod=2022&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_life_exp_time = oecd_to_df(xml, data_life_exp)

reshaped_df_life_exp_time = df_life_exp_time.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    columns='SEX',
    values='ObsValue',
    aggfunc='first'
)

reshaped_df_life_exp_time = reshaped_df_life_exp_time.rename(columns={'F' : 'Female', 'M' : 'Male', '_T' : 'Life_Expectancy'})

data_population = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'AGE' : [],
    'SEX' : [],
    'TIME_HORIZ' : [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_POPULATION%40DF_POP_HIST,1.0/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+G20+EU27+OECD..PS._T._T.?startPeriod=1980&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_population_time = oecd_to_df(xml, data_population)

reshaped_df_population_time = df_population_time.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    columns='AGE',
    values='ObsValue',
    aggfunc='first'
)

reshaped_df_population_time = reshaped_df_population_time.rename(columns={'_T' : 'Population'})

data_wages = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_EARNINGS%40AV_AN_WAGE,1.0/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+OECD..USD_PPP....?startPeriod=1991&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_wage_time = oecd_to_df(xml, data_wages)

reshaped_df_wage_time = df_wage_time.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    values='ObsValue',
    aggfunc='first'
)

reshaped_df_wage_time = reshaped_df_wage_time.rename(columns={'ObsValue' : 'Mean_Annual_Wages'})

data_wages_extreme = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'ObsValue': []
}


url = "https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_EARNINGS@PAY_INCIDENCE,1.0/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+POL+PRT+SVK+SVN+ESP+CHE+TUR+GBR+USA+OECD.LP_I+HP_I....._T?startPeriod=1991&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_wage_extreme_time = oecd_to_df(xml, data_wages_extreme)

reshaped_df_wage_extreme_time = df_wage_extreme_time.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    columns='MEASURE',
    values='ObsValue',
    aggfunc='first'
)

reshaped_df_wage_extreme_time = reshaped_df_wage_extreme_time.rename(columns={"HP_I" : "High_Pay", "LP_I" : "Low_Pay"})

data_wbs_time = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.WISE.WDP,DSD_HSL@DF_HSL_CWB,1.0/AUT+BEL+COL+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+AUS+CAN+JPN+GBR.11_1_DEP+11_1.._T._T._T.?startPeriod=2009&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_wbs_time = oecd_to_df(xml, data_wbs_time)

reshaped_df_wbs_time = df_wbs_time.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    columns='MEASURE',
    values='ObsValue',
    aggfunc='first'
)

reshaped_df_wbs_time = reshaped_df_wbs_time.rename(columns={'11_1' : 'WBS', '11_1_DEP' : 'WBSu5'})

data_gdp_time = {
    'TIME_PERIOD': [],
    'REF_AREA': [],
    'MEASURE': [],
    'UNIT_MEASURE': [],
    'ObsValue': []
}

url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_I,1.0/A.AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA.B1GQ+B1GQ_R_GR.PC+USD_PPP.?startPeriod=1980&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url)
xml = response.text
df_gdp_time = oecd_to_df(xml, data_gdp_time)


reshaped_df_gdp_time = df_gdp_time.pivot_table(
    index=['TIME_PERIOD', 'REF_AREA'],
    columns='MEASURE',
    values='ObsValue',
    aggfunc='first'
)

reshaped_df_gdp_time = reshaped_df_gdp_time.rename(columns={"B1GQ" : "GDP", "B1GQ_R_GR" : "GDP_Growth_Rate"})

In [ ]:
df_corr = pd.merge(reshaped_df_trust, reshaped_df_gender, on='REF_AREA', how='left')

to_merge = reorderd_df_demo.iloc[:, -2:]
df_corr = pd.merge(df_corr, to_merge, on='REF_AREA', how='left')

to_merge = reshaped_df_life_exp_time['Life_Expectancy']
df_corr = pd.merge(df_corr, to_merge, on=['REF_AREA', 'TIME_PERIOD'], how='left')

df_corr = pd.merge(df_corr, reshaped_df_population_time, on=['REF_AREA', 'TIME_PERIOD'], how='left')
df_corr = pd.merge(df_corr, reshaped_df_wage_time, on=['REF_AREA', 'TIME_PERIOD'], how='left')
to_merge = reshaped_df_wage_extreme_time['Low_Pay']
df_corr = pd.merge(df_corr, to_merge, on=['REF_AREA', 'TIME_PERIOD'], how='left')
df_corr = pd.merge(df_corr, reshaped_df_wbs_time, on=['REF_AREA', 'TIME_PERIOD'], how='left')
to_merge = reshaped_df_gdp_time['GDP_Growth_Rate']
df_corr = pd.merge(df_corr, to_merge, on=['REF_AREA', 'TIME_PERIOD'], how='left')

df_corr = df_corr.set_index(['TIME_PERIOD', 'REF_AREA'])

In [ ]:
non_numeric_columns = df_corr.select_dtypes(exclude=[np.number]).columns

# Converting non-numeric columns to float
for column in non_numeric_columns:
    df_corr[column] = df_corr[column].astype(float)

In [ ]:
df_corr.to_csv("faktentabelle.csv")

In [ ]:
cor = df_corr.corr()
cor.to_csv("cor.csv")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Assume `df` is your DataFrame
df = df_corr

# Identify categorical columns that need encoding and numerical columns that need imputation
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
numerical_cols = df.select_dtypes(exclude=['object', 'category']).columns.drop('TRUST_NG')

# Preprocessing for numerical data: imputation
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data: imputation + one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

# Separate target from predictors

df = df.dropna(subset=['TRUST_NG'])
X = df.drop('TRUST_NG', axis=1)
y = df['TRUST_NG']


# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_val)

# Evaluate the model
score = mean_squared_error(y_val, preds, squared=False)  # RMSE
mae = mean_absolute_error(y_val, preds)  # RMSE
print('RMSE:', score)
print('MAE: ', mae)


RMSE: 0.6060199987756804
MAE:  0.4904529068563962
